<a href="https://colab.research.google.com/github/anirudh201098/Object-detection-with-Yolo-models/blob/main/Yolo%20models/Yolov_3_Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://miro.medium.com/max/3802/1*d4Eg17IVJ0L41e7CTWLLSg.png)

#Pre-requisites :
A) YOLO(v3) requires image annotations in .txt extension in the following format: 


```
<object-class> <x> <y> <height> <width>
For example if tiger is object class 0 in an image "img1.png" 
then annotation file will also have the name img1.txt 
and annotation in the file will be this for one bounding box:

0 0.295491 0.631188 0.589920 0.502829


```


B) All the images and respective annotation file should be present in the same directory. 



 **Introduction**

The main goal of this challenge is to recognize objects from a number of visual object classes in realistic scenes (i.e. not pre-segmented objects). It is fundamentally a supervised learning learning problem in that a training set of labelled images is provided. The twenty object classes that have been selected are:

    Person: person
    Animal: bird, cat, cow, dog, horse, sheep
    Vehicle: aeroplane, bicycle, boat, bus, car, motorbike, train
    Indoor: bottle, chair, dining table, potted plant, sofa, tv/monitor


**PSACAL VOC 2012 dataset: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html**


**1. Mount the drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. List the files and directories at current path**

In [2]:
%ls


drive/  sample_data/


**3. Move to the folder where dataset and main files are present**

In [3]:
%cd /content/drive/My\ Drive/Research_Internship/Yolo-3

/content/drive/My Drive/Research_Internship/Yolo-3


**4. List the files and directories at current path**

In [4]:
%ls 

darknet/


**5. Clone darknet** (uncomment when running it for first time after clone comment it again)

In [ ]:
# !git clone https://github.com/AlexeyAB/darknet/  

In [ ]:

%ls

darknet/


In [5]:
%cd darknet 

/content/drive/My Drive/Research_Internship/Yolo-3/darknet


**6. List the files and directories at current path**

In [6]:
!ls

 3rdparty		       darknet.py	       results
 backup			       darknet_video.py        scripts
 bad.list		       data		       sed5kQqMR
 build			       dataset		       sedAZqhW0
 build.ps1		      'dataset (1)'	       sedCjc4yQ
 build.sh		       image_yolov2.sh	       sedFqktGh
 cfg			       image_yolov3.sh	       sedGZtkyD
 chart.png		       include		       sedH54eZw
 chart_yolov3-voc-custom.png   json_mjpeg_streams.sh   sedIMHa1g
 cmake			       LICENSE		       sedMaTwYm
 CMakeLists.txt		       Makefile		       src
 darknet		       net_cam_v3.sh	       video_v2.sh
 darknet53.conv.74	       obj		       video_yolov3.sh
 darknet53.conv.74.1	       predictions.jpg
 DarknetConfig.cmake.in        README.md


**7. Activate GPU, CUDNN and OPENCV in makefile** (Uncomment it for first time run then again comment it)

In [7]:
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile

**8. Build executable code using make** (Uncomment it for first time run after that again comment below make command)

In [ ]:
 !make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130:10: note: ...thi

**9. Change the permission for dakrnet file**

In [8]:
!chmod +x ./darknet 

**10. Downloading the Imagenet weights for YOLO V3**

In [ ]:
#!wget https://pjreddie.com/media/files/darknet53.conv.74

**11. Setting up the configuration**

**(A) Altering .cfg file**

It contains the model architecture information.

**Download yolov3-voc-custom.cfg file from cfg folder of darknet**



**Open the file using any editor and do following changes:**
*   change line batch to batch=64
*   change line subdivisions to subdivisions=16
*   height =608, width=608 or any value multiple of 32
*   change line max_batches to 40000(classes*2000 i.e 20x2000)
*   change line steps to 80% and 90% of max_batches, f.e. steps=32000,36000
* change line classes=80 to your number of objects in each of 3 [yolo]-layers, f. e. classes=20
*  change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers, i. e. filters=75

**(B) Creating .names file**

*   It contains the class information
*   Create a new file in your editor named it as voc.names
*   In the file write the name of classes(each class name in new line)

```
voc.names:

aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
```

**(C) Creating .data file**

It contains information about path of training file, testing file, class name file and path where to store the trained weights 

```
voc.data
classes= 20
train  = data/voc/train.txt    => Train file path
valid  = data/test.txt  => Test file path
names = data/voc.names => Classes file path
backup = backup/   => Weights will be saved after every 100 epochs in this directory.
```
Make sure you save the train and test .txt  files in data folder of darknet

**12. Before you proeed ahead upload the modifiled files:**

*   Upload voc.names and voc.data inside data folder of darknet
*   Upload voc.cfg inside cfg folder of darknet

**13. Install dos2unix library to change files as per unix compatabiity**

In [9]:
!sudo apt-get install dos2unix 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 0s (4,621 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 144628 files and directories curr

**14. Convert the uploaded file in unix format**

If you get an error here, then recheck you paths for all the files and paste them accordingly.


In [10]:
 !dos2unix /content/drive/My\ Drive/Research_Internship/Yolo-3/darknet/data/train.txt
 !dos2unix /content/drive/My\ Drive/Research_Internship/Yolo-3/darknet/data/test.txt
 !dos2unix /content/drive/My\ Drive/Research_Internship/Yolo-3/darknet/data/voc.data
 !dos2unix /content/drive/My\ Drive/Research_Internship/Yolo-3/darknet/data/voc.names
 !dos2unix /content/drive/My\ Drive/Research_Internship/Yolo-3/darknet/cfg/yolov3-voc-custom.cfg

dos2unix: converting file /content/drive/My Drive/Research_Internship/Yolo-3/darknet/data/train.txt to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/Yolo-3/darknet/data/test.txt to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/Yolo-3/darknet/data/voc.data to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/Yolo-3/darknet/data/voc.names to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/Yolo-3/darknet/cfg/yolov3-voc-custom.cfg to Unix format...


In [ ]:
#!./darknet detector train data/voc.data cfg/yolov3-voc-custom.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3-voc-custom
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   3 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   5 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
   6 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
   7 conv    128       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x 128 3.407 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, o

**15. Training dataset** (When training the model from pre-trained weights use below command )

Since I have already trained the PASCAL VOC 2012 train data for 38000 epochs, I am using the following command.

If you are training the data for the first time, then change the code to 
```
!./darknet detector train data/voc.data cfg/yolov3-voc-custom.cfg darknet53.conv.74 -dont_show
```

If you have dataset which has similar classes to PASCAL VOC 2012 dataset, you can use pre-trained weights from the following link: **https://drive.google.com/file/d/1-Bd76ZWfzlYP44LDO1F1lD5RQsihMZ45/view?usp=sharing**

In [ ]:
 !./darknet detector train data/voc.data cfg/yolov3-voc-custom.cfg backup/yolov3-voc-custom_38000.weights -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3-voc-custom
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   3 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   5 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
   6 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
   7 conv    128       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x 128 3.407 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, o

**16. Calculate the Mean average Precision of the trained model**

In [ ]:
!./darknet detector map data/voc.data cfg/yolov3-voc-custom.cfg backup/yolov3-voc-custom_last.weights -dont_show 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   3 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   5 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
   6 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
   7 conv    128       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x 128 3.407 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 152 x 152 

**17. Predicting bounding boxes for new image:**

The predicted image will be saved in ./darknet folder with the name predictions.jpg

In [ ]:
!./darknet detector test data/voc.data cfg/yolov3-voc-custom.cfg backup/yolov3-voc-custom_40000.weights data/dog.jpg -dont_show 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   3 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   5 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
   6 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
   7 conv    128       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x 128 3.407 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 152 x 152 

**18. Prediction for videos**

In [12]:
!./darknet detector demo data/voc.data cfg/yolov3-voc-custom.cfg backup/yolov3-voc-custom_40000.weights data/traffic.mp4 -out_filename yolo3_pred.avi -dont_show

Streaming output truncated to the last 5000 lines.
FPS:36.5 	 AVG_FPS:36.1

 cvWriteFrame 
Objects:

person: 81% 
car: 100% 
car: 100% 
car: 100% 
car: 99% 
car: 99% 
car: 99% 
car: 99% 
car: 99% 
car: 99% 
car: 98% 
car: 98% 
car: 98% 
car: 97% 
car: 96% 
car: 94% 
car: 90% 
car: 87% 
car: 78% 
car: 30% 

FPS:36.4 	 AVG_FPS:36.1

 cvWriteFrame 
Objects:

person: 66% 
person: 36% 
car: 100% 
car: 100% 
car: 100% 
car: 100% 
car: 100% 
car: 100% 
car: 99% 
car: 99% 
car: 99% 
car: 98% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 89% 
car: 87% 
car: 75% 
car: 70% 
car: 39% 

FPS:36.2 	 AVG_FPS:36.1

 cvWriteFrame 
Objects:

person: 63% 
person: 52% 
person: 27% 
car: 100% 
car: 100% 
car: 100% 
car: 99% 
car: 99% 
car: 99% 
car: 99% 
car: 99% 
car: 99% 
car: 98% 
car: 98% 
car: 97% 
car: 97% 
car: 97% 
car: 97% 
car: 95% 
car: 75% 
car: 66% 

FPS:36.3 	 AVG_FPS:36.1

 cvWriteFrame 
Objects:

person: 41% 
person: 26% 
car: 100% 
car: 100% 
car: 100% 
car: 100% 
car: 100% 
car: 99% 
car: 